Importamos librerias a usar

In [ ]:
import ast
import json
import pandas as pd

Al tratar de abrir archivo output_steam_games.json posee demasiados errores así que se procede a separar palabras, se cambia  comillas simples por dobles ,se compone las separaciones y gradamos el resultado en un archivó para los datos y otro para los errores que no se pudieron corregir.


In [ ]:
entries = []
errores = []

# Función para manejar NaN, Infinity, -Infinity
def handle_nan_constant(value):
    if value in ('NaN', 'Infinity', '-Infinity'):
        return None  # No usar 'Desconocido', simplemente no llenar el valor
    return value

with open('/content/output_steam_games.json', 'r', encoding='utf-8', errors='ignore') as f:
    data = f.read()
data = data.replace('}{', '}\n{')

lines = data.strip().split('\n')

for line_number, line in enumerate(lines, 1):
    line = line.strip()
    if not line:
        continue
    try:
        obj = json.loads(line, parse_constant=handle_nan_constant)
        publisher = obj.get('publisher', '')
        genres = obj.get('genres', [])
        app_name = obj.get('app_name', '')
        title = obj.get('title', '')
        url = obj.get('url', '')
        release_date = obj.get('release_date', '')
        tags = obj.get('tags', [])
        reviews_url = obj.get('reviews_url', '')
        specs = obj.get('specs', [])
        price = obj.get('price', '')
        early_access = obj.get('early_access', False)
        game_id = obj.get('id', '')
        developer = obj.get('developer', '')

        # Añadir la información procesada a la lista de entradas
        entries.append({
            'publisher': publisher,
            'genres': genres,
            'app_name': app_name,
            'title': title,
            'url': url,
            'release_date': release_date,
            'tags': tags,
            'reviews_url': reviews_url,
            'specs': specs,
            'price': price,
            'early_access': early_access,
            'id': game_id,
            'developer': developer
        })
    except Exception as e:
        # Si hay un error, registrar la línea en el archivo de errores
        print(f'Error al procesar la línea {line_number}: {e}')
        errores.append({
            'line_number': line_number,
            'error': str(e),
            'line': line
        })

# Guardar las entradas válidas en un archivo JSON
with open('/content/output_steam_games_coregidos.json', 'w', encoding='utf-8') as f:
    json.dump(entries, f, ensure_ascii=False, indent=4)

# Guardar las líneas problemáticas en un archivo separado para revisarlas después
with open('/content/output_steam_games_errores.json', 'w', encoding='utf-8') as f:
    json.dump(errores, f, ensure_ascii=False, indent=4)

print('Los datos válidos han sido extraídos y guardados en "output_steam_games_coregidos.json".')
print('Las líneas problemáticas han sido registradas en "output_steam_games_errores.json".')


Error al procesar la línea 93288: Unterminated string starting at: line 1 column 15 (char 14)
Error al procesar la línea 93289: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Error al procesar la línea 93290: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Los datos válidos han sido extraídos y guardados en "output_steam_games_coregidos.json".
Las líneas problemáticas han sido registradas en "output_steam_games_errores.json".


3 filas del archivo no pudieron ser coregidas en el paso anterior prosedemos a coregirlas manualmente y luego sumarlas a los datos corregidos


In [ ]:
import json

# Datos nuevos a añadir
new_data = [
    {
        "publisher": "CodeAtch",
        "genres": ["Action", "Adventure", "Indie", "RPG", "Simulation"],
        "app_name": "Reign Of Kings",
        "title": "Reign Of Kings",
        "url": "http://store.steampowered.com/app/344760/Reign_Of_Kings/",
        "release_date": "2015-12-15",
        "tags": ["Survival", "Open World", "Multiplayer", "Medieval", "Crafting", "Building", "PvP", "Sandbox", "Action", "Adventure", "RPG", "Online Co-Op", "Realistic", "Gore", "Simulation", "Co-op", "Indie", "Dark Fantasy", "Dark Humor", "FPS"],
        "reviews_url": "http://steamcommunity.com/app/344760/reviews/?browsefilter=mostrecent&p=1",
        "specs": ["Multi-player"],
        "price": 19.99,
        "early_access": False,
        "id": "344760",
        "developer": "CodeAtch"
    },
    {
        "publisher": "Dovetail Games - Flight",
        "genres": ["Simulation"],
        "app_name": "FSX: Steam Edition - Piper Aztec Add-On",
        "title": "FSX: Steam Edition - Piper Aztec Add-On",
        "url": "http://store.steampowered.com/app/379568/FSX_Steam_Edition__Piper_Aztec_AddOn/",
        "release_date": "2015-12-16",
        "tags": ["Simulation"],
        "reviews_url": "http://steamcommunity.com/app/379568/reviews/?browsefilter=mostrecent&p=1",
        "specs": ["Single-player", "Multi-player", "Downloadable Content", "Partial Controller Support"],
        "price": 19.99,
        "early_access": False,
        "id": "379568",
        "developer": "Aeroplane Heaven"
    },
    {
        "publisher": "Yeti Studio©",
        "genres": ["Adventure", "Indie", "Early Access"],
        "app_name": "AWAKE",
        "title": "AWAKE",
        "url": "http://store.steampowered.com/app/421110/AWAKE/",
        "release_date": "2015-12-16",
        "tags": ["Early Access", "Adventure", "Indie", "Survival"],
        "reviews_url": "http://steamcommunity.com/app/421110/reviews/?browsefilter=mostrecent&p=1",
        "specs": ["Single-player", "Partial Controller Support"],
        "price": 4.99,
        "early_access": True,
        "id": "421110",
        "developer": "Diios"
    }
]
file_path = '/content/output_steam_games_coregidos.json'

# Intentamos leer el archivo existente
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        existing_data = json.load(file)
        # Verificamos si el archivo tiene una lista
        if not isinstance(existing_data, list):
            print("El archivo no contiene una lista. Inicializando una lista vacía.")
            existing_data = []
except (FileNotFoundError, json.JSONDecodeError):
    print("El archivo no existe o tiene un error de formato. Inicializando una lista vacía.")
    existing_data = []

# Añadir los nuevos datos a los datos existentes
existing_data.extend(new_data)

# Guardar los datos actualizados en el archivo correcto
with open(file_path, 'w', encoding='utf-8') as file:
    json.dump(existing_data, file, ensure_ascii=False, indent=4)

print(f"Datos agregados correctamente al archivo '{file_path}'.")



Datos agregados correctamente al archivo '/content/output_steam_games_coregidos.json'.


Una ves el documento esta  correjiodo trsformamos a csv  renombranos columnas para una mejor comprencion i cambiamos formato de fecha a una mas compatible con python y cambiamos espacios vacios por Nan o desconocido segun coresponda

In [ ]:
import json
import csv
from datetime import datetime

def format_date(date_str):
    if not date_str:
        return "NaN"
    try:
        return datetime.strptime(date_str, "%Y-%m-%d").strftime("%d-%m-%Y")
    except ValueError:
        return "NaN"

json_file_path = '/content/output_steam_games_coregidos.json'
csv_file_path = '/content/output_steam_games.csv'

# Leer el archivo JSON corregido
try:
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
except FileNotFoundError:
    print(f"El archivo JSON '{json_file_path}' no se encontró.")
    data = []
except json.JSONDecodeError:
    print(f"Error al decodificar el archivo JSON '{json_file_path}'.")
    data = []

# Encabezados del CSV
fieldnames = [
    "Nombre_App", "Título_Juego", "Desarrollador", "Editor", "Géneros_Juego",
    "Etiquetas_Juego", "Fecha_Lanzamiento", "Precio", "Acceso_Anticipado",
    "Especificaciones_Juego", "URL_Tienda"
]

# Abrir el archivo CSV para escribir los datos
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Procesar cada entrada de datos del archivo JSON
    for game in data:
        writer.writerow({
            "Nombre_App": game.get("app_name", "NaN"),
            "Título_Juego": game.get("title", "NaN"),
            "Desarrollador": game.get("developer", "Desconocido") if game.get("developer") else "Desconocido",
            "Editor": game.get("publisher", "NaN"),
            "Géneros_Juego": ", ".join(game.get("genres", []) if isinstance(game.get("genres"), list) else ["NaN"]),
            "Etiquetas_Juego": ", ".join(game.get("tags", []) if isinstance(game.get("tags"), list) else ["NaN"]),
            "Fecha_Lanzamiento": format_date(game.get("release_date", "")),  # Pasa cadena vacía si no hay fecha
            "Precio": game.get("price", "NaN"),
            "Acceso_Anticipado": "Sí" if game.get("early_access", False) else "No",
            "Especificaciones_Juego": ", ".join(game.get("specs", []) if isinstance(game.get("specs"), list) else ["NaN"]),
            "URL_Tienda": game.get("url", "NaN")
        })

print(f"Archivo CSV creado exitosamente en '{csv_file_path}'.")


Archivo CSV creado exitosamente en '/content/output_steam_games.csv'.


EDA

Al no haber errores en el archivo csv  comenzamos el EDA cargando  la información en un data frame y visualizamos las primeras líneas

In [ ]:
import pandas as pd

df = pd.read_csv(file_path, low_memory=False)


file_path = '/content/output_steam_games.csv'

df.head()

,Nombre_App,Título_Juego,Desarrollador,Editor,Géneros_Juego,Etiquetas_Juego,Fecha_Lanzamiento,Precio,Acceso_Anticipado,Especificaciones_Juego,URL_Tienda
0,NaN,NaN,Desconocido,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN
1,NaN,NaN,Desconocido,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN
2,NaN,NaN,Desconocido,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN
3,NaN,NaN,Desconocido,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN
4,NaN,NaN,Desconocido,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN


Realizamos un resumen estadístico delos datos numéricos, hacemos un resumen de cantidad de filas y columnas , obtenemos información del archivo y revisamos la cantidad de datos nulos por columna

In [ ]:
df.describe(include='object')
print(f"Dimensiones del DataFrame: {df.shape}")

df.info()

print("Valores nulos por columna:")
print(df.isnull().sum())

Dimensiones del DataFrame: (120450, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120450 entries, 0 to 120449
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Nombre_App              32138 non-null   object
 1   Título_Juego            30090 non-null   object
 2   Desarrollador           120450 non-null  object
 3   Editor                  24069 non-null   object
 4   Géneros_Juego           28857 non-null   object
 5   Etiquetas_Juego         31977 non-null   object
 6   Fecha_Lanzamiento       29788 non-null   object
 7   Precio                  30763 non-null   object
 8   Acceso_Anticipado       120450 non-null  object
 9   Especificaciones_Juego  31470 non-null   object
 10  URL_Tienda              32140 non-null   object
dtypes: object(11)
memory usage: 10.1+ MB
Valores nulos por columna:
Nombre_App                88312
Título_Juego              90360
Desarrollador              

Después de una revisión visual se observa que las 88311 primeras filas están vacías y las boro

In [ ]:

df_cleaned = df.iloc[88311:].copy()

print(f"Número de filas después de eliminar las primeras 88,311 filas vacías: {len(df_cleaned)}")
df_cleaned.to_csv('/content/output_steam_games.csv', index=False, encoding='utf-8')

print("El DataFrame limpio ha sido guardado en el nuevo archivo CSV.")


Número de filas después de eliminar las primeras 88,311 filas vacías: 32139
El DataFrame limpio ha sido guardado en el nuevo archivo CSV.


In [ ]:
df.describe(include='object')
print(f"Dimensiones del DataFrame: {df.shape}")

df.info()

print("Valores nulos por columna:")
print(df.isnull().sum())

Dimensiones del DataFrame: (120450, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120450 entries, 0 to 120449
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Nombre_App              32138 non-null   object
 1   Título_Juego            30090 non-null   object
 2   Desarrollador           120450 non-null  object
 3   Editor                  24069 non-null   object
 4   Géneros_Juego           28857 non-null   object
 5   Etiquetas_Juego         31977 non-null   object
 6   Fecha_Lanzamiento       29788 non-null   object
 7   Precio                  30763 non-null   object
 8   Acceso_Anticipado       120450 non-null  object
 9   Especificaciones_Juego  31470 non-null   object
 10  URL_Tienda              32140 non-null   object
dtypes: object(11)
memory usage: 10.1+ MB
Valores nulos por columna:
Nombre_App                88312
Título_Juego              90360
Desarrollador              